<a href="https://colab.research.google.com/github/E-tech-coder/DataScienceCapstoneProject/blob/Elena/RuleBased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/E-tech-coder/DataScienceCapstoneProject.git

Cloning into 'DataScienceCapstoneProject'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 31 (delta 15), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 456.85 KiB | 5.02 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
import pandas as pd

df = pd.read_csv("/content/DataScienceCapstoneProject/linkedin_experience_annotated.csv").drop(columns="linkedin").set_index("person_id").reset_index()

In [3]:
df.head(20)

,person_id,organization,position,startDate,endDate,status,department,seniority
0,0,Depot4Design GmbH,Prokurist,2019-08,NaN,ACTIVE,Other,Management
1,0,Depot4Design GmbH,CFO,2019-07,NaN,ACTIVE,Other,Management
2,0,Depot4Design GmbH,Betriebswirtin,2019-07,NaN,ACTIVE,Other,Professional
3,0,Depot4Design GmbH,Prokuristin,2019-07,NaN,ACTIVE,Other,Management
4,0,Depot4Design GmbH,CFO,2019-07,NaN,ACTIVE,Other,Management
5,0,Nagel Car Group,Buchhalterin,2000-05,2019-06,INACTIVE,Other,Professional
6,1,Computer Solutions,Solutions Architect,2024-03,NaN,ACTIVE,Information Technology,Professional
7,1,Computer Solutions,Senior Network Engineer,2019-07,2024-03,INACTIVE,Information Technology,Senior
8,1,Texas A&M University-Corpus Christi,Manager of Network Services,2017-02,2019-07,INACTIVE,Information Technology,Professional
9,1,Texas A&M University-Corpus Christi,Infrastructure Administrator II,2015-06,2017-02,INACTIVE,Information Technology,Professional


In [5]:
dpt = pd.read_csv("/content/DataScienceCapstoneProject/department-v2.csv")

In [6]:
dpt['label'].drop_duplicates()

,label
0,Marketing
1,Project Management
171,Administrative
202,Business Development
374,Consulting
476,Human Resources
487,Information Technology
2609,Other
2613,Purchasing
2636,Sales


In [55]:
import re

BOW = dict()

for label in dpt['label'].drop_duplicates() :
  x = []
  for text in dpt[dpt["label"] == label]["text"]:
    tokens = re.split(r"[ ./&-()|-]", text.lower().strip())
    tokens = [t for t in tokens if t]
    x.extend(tokens)
  BOW[label] = list(set(x))


In [57]:
label

'Customer Support'

In [29]:
df["WordAppearance"] = 0
df["PredictedPosition_BOW"] =""
df["position_words"] = df["position"].apply(lambda x : re.split(r"[ /&-]", x.lower().strip()))

In [30]:
for label, BOW_words in BOW.items():
  bow_set = set(BOW_words)
  # Count how many words of the position appear in the bag of words of the label
  for idx,words in df["position_words"].items():
    count=sum(word in bow_set for word in words)
        # Update prediction if the current word count is greater than the previous one
    if count > df.at[idx, "WordAppearance"] :
      df.at[idx, "WordAppearance"] = count
      df.at[idx, "PredictedPosition_BOW"] = label

In [32]:
df.to_csv("/content/DataScienceCapstoneProject/2BOW-Position-Predict.csv")

In [11]:
# The performance using only bag of words for classification is very terrible.
# Let's try TFIDF method.

In [58]:
corpus = []
labels = []

for key, value in BOW.items():
  labels.append(key)
  corpus.append(value)


In [59]:
len(corpus)

11

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [" ".join(doc) for doc in corpus]
titles = labels

vectorizer = TfidfVectorizer()
vector = vectorizer.fit_transform(corpus)

In [ ]:
dict(zip(vectorizer.get_feature_names_out(), vector.toarray()[0]))

In [70]:
tfidf_df = pd.DataFrame(vector.toarray(), index = titles, columns = vectorizer.get_feature_names_out())

In [ ]:
tfidf_df.iloc[:, 50:80]

In [ ]:
# for label, BOW_words in BOW.items():
#   bow_set = set(BOW_words)
#   # Count how many words of the position appear in the bag of words of the label
#   for idx,words in df["position_words"].items():
#     count=sum(word in bow_set for word in words)
#         # Update prediction if the current word count is greater than the previous one
#     if count > df.at[idx, "WordAppearance"] :
#       df.at[idx, "WordAppearance"] = count
#       df.at[idx, "PredictedPosition_BOW"] = label